In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install kmodes

In [ ]:
#@title Import
import pandas as pd
import pandas as pd
import random
import numpy as np
import datetime
import time
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
import scipy
from  kmodes.kprototypes import KPrototypes
from sklearn.cluster import KMeans
from sklearn.preprocessing import PowerTransformer
from sklearn.cluster import KMeans

In [ ]:
#@title Reading the data
df = pd.read_csv("/content/drive/My Drive/Thesis-data/CH3_F_10000.csv",low_memory=False)
df['time d'] = pd.to_datetime(df['time d']).dt.hour

In [ ]:
df.head()

,ID,Date,Day_name,d,time d,duration,distance,stations
0,7871296,2020-01-21,Tuesday,123.0,6,13,0.0,"(123.0, 123.0)"
1,7871296,2020-01-21,Tuesday,92.0,19,12,0.0,"(92.0, 92.0)"
2,7871296,2020-01-22,Wednesday,123.0,6,12,0.0,"(123.0, 123.0)"
3,7871296,2020-01-22,Wednesday,92.0,18,12,0.0,"(92.0, 92.0)"
4,7871296,2020-01-23,Thursday,123.0,6,8,0.0,"(123.0, 123.0)"


In [ ]:
#@title Clustering the actvivties into 3 major clusters
dataee = df.copy()
dataee = df.iloc[:,[4,5]]
dataee.head()
##
kprot_data = dataee.copy()
for c in dataee.select_dtypes(exclude='object').columns:
    pt = PowerTransformer()
    kprot_data[c] =  pt.fit_transform(np.array(kprot_data[c]).reshape(-1, 1))
##
kprot_data['duration'] = kprot_data['duration'] * 2
##
kproto = KMeans(n_clusters = 3, init ='k-means++')
clusters = kproto.fit_predict(kprot_data)

## KProto algorithm
# kproto = KPrototypes(n_clusters=3,init='Cao', n_jobs=50)
# categorical_columns= [0]
# clusters = kproto.fit_predict(dataee, categorical=categorical_columns)
##
cluster_lables = kproto.labels_
kprot_data['z'] = cluster_lables
dataee['z'] = cluster_lables
df['z'] = cluster_lables

In [ ]:
#@title Cluster description (attributes average)
dataT10 = df
print('Duration')
print("%.2f" % np.average(dataT10[dataT10['z']==0]['duration']))
print("%.2f" % np.average(dataT10[dataT10['z']==1]['duration']))
print("%.2f" % np.average(dataT10[dataT10['z']==2]['duration']))
####
print('Time')
print("%.2f" % np.average(dataT10[dataT10['z']==0]['time d']))
print("%.2f" % np.average(dataT10[dataT10['z']==1]['time d']))
print("%.2f" % np.average(dataT10[dataT10['z']==2]['time d']))

In [ ]:
#@title Finding the home location
datauc = df.copy()
datauc['indx'] = datauc.index

data_home= datauc[datauc['z']==2]
df_home = data_home.groupby(['ID', 'd']).size().reset_index(name="Time")
df_home = df_home.sort_values('Time')
df_home = pd.DataFrame(df_home)
datauc['home_location'] = pd.Series(dtype='object')

def home_location(x):

    datauc.at[x['indx'],'home_location'] =  df_home[df_home['ID']== x['ID']]['d'].unique().tolist()
    return datauc

datauc.apply(home_location, axis=1)

print(len(datauc[datauc['z']==1]))

def other_home(x):

  if x['d'] in x['home_location']:
    datauc.at[x['indx'],'z']=102
    df.at[x['indx'],'z']=102
  return df

datauc[datauc['z']==1].apply(other_home,axis=1)
print(len(datauc[datauc['z']==1]))

In [ ]:
#@title Frequnecy
data = datauc.copy()
df = data.groupby(["z", "ID", 'd']).size().reset_index(name="Time")
df = df.sort_values('Time')
data['index'] = data.index


def frequency(x):

    z = x['z']
    idd = x['ID']
    d = x['d']
    index = x['index']

    data.at[index, 'frequency'] = df[ (df['z'] == z) & (df['ID'] == idd) & (df['d'] == d) ]['Time'].values[0]

    return data

data.apply(frequency, axis=1)
data_all = data.copy()

In [ ]:
data_all.to_csv("/content/drive/My Drive/Thesis-data/data_all_with_frequency_10000_1398-11_extra.csv",index=False)

# Calculating the frequency for "other" activities

In [ ]:
data_all  = pd.read_csv('/content/drive/My Drive/Thesis-data/data_all_with_frequency_10000_1399-11_extra.csv',low_memory=False)

In [ ]:
#@title Choosing the "other" cluster
data_other = data_all[data_all['z']== 1]

In [ ]:
#@title clustering the "other" cluster intor 4 major groups for calculating the frequency
dataee = data_other.iloc[:,[4,5]]
## Scalar ##
kprot_data = dataee.copy()
for c in dataee.select_dtypes(exclude='object').columns:
    pt = PowerTransformer()
    kprot_data[c] =  pt.fit_transform(np.array(kprot_data[c]).reshape(-1, 1))
## KMean ##
kproto = KMeans(n_clusters = 4, init ='k-means++')
clusters = kproto.fit_predict(kprot_data)
## Cluster labels
cluster_lables = kproto.labels_
dataee['z'] = cluster_lables
data_other['z'] = cluster_lables

In [ ]:
#@title Cluster description (attributes average)
dataT10 = dataee
print('Duration')
print("%.2f" % np.average(dataT10[dataT10['z']==0]['duration']))
print("%.2f" % np.average(dataT10[dataT10['z']==1]['duration']))
print("%.2f" % np.average(dataT10[dataT10['z']==2]['duration']))
print("%.2f" % np.average(dataT10[dataT10['z']==3]['duration']))

# #
print('Time')
print("%.2f" % np.average(dataT10[dataT10['z']==0]['time d']))
print("%.2f" % np.average(dataT10[dataT10['z']==1]['time d']))
print("%.2f" % np.average(dataT10[dataT10['z']==2]['time d']))
print("%.2f" % np.average(dataT10[dataT10['z']==3]['time d']))

In [ ]:
#@title Calculating the activity frequency
## Frquency ##
data = data_other
df = data.groupby(["z", "ID", 'd']).size().reset_index(name="Time")
df = df.sort_values('Time')
data['index'] = data.index

def frequency(x):

    z = x['z']
    idd = x['ID']
    d = x['d']
    index = x['index']

    data.at[index, 'frequency'] = df[ (df['z'] == z) & (df['ID'] == idd) & (df['d'] == d) ]['Time'].values[0]

    return data
    
data.apply(frequency, axis=1)
print(data.head())

In [ ]:
data.to_csv("/content/drive/My Drive/Thesis-data/CH3_F_10000_other(F)_1398-11_extra.csv",index=False)